In [ ]:
!pip install spacy


In [ ]:
!spacy download pt_core_news_sm

In [ ]:
import spacy
from spacy import displacy
import pandas as pd
import re

In [ ]:
nlp = spacy.load('pt_core_news_sm')

In [ ]:
datasetDiscursos = pd.read_csv('/content/discurso_df_sp.csv')


In [ ]:
def remover_padroes(texto):
    # Cria uma string de padrão regular que representa padrões específicos de texto que desejamos remover.
    # Este padrão busca por várias variantes de 'O SR.' seguidas por diferentes sequências de texto até certos delimitadores.
    padrao = r'O SR\. .*?Sem revisão do orador\.\) - |O SR\. .*?\) - |O SR\. .*?Pronuncia o seguinte discurso\.\)'
    return re.sub(padrao, '', texto)
# Aplica a função em cada discurso
datasetDiscursos['discurso_trat'] = datasetDiscursos['transcricao'].apply(remover_padroes)

In [ ]:
deputados = pd.read_csv('/content/deputados.csv')


In [ ]:
debate_dep.shape

(252, 7)

In [ ]:
debate_dep = pd.merge(datasetDiscursos[['id_Deputado','keywords','discurso_trat']],deputados[["id",'nome','siglaPartido','siglaUf']] , left_on='id_Deputado', right_on='id', how="inner")

In [ ]:
texto = debate_dep['discurso_trat']

In [ ]:
debate_dep['discurso_trat'][138]

'Sr. Presidente, Sras. e Srs. Parlamentares, cidadãos que assistem à sessão pela TV Câmara, aproveito a oportunidade para convidar a sociedade e os Parlamentares presentes para, na próxima quinta-feira, dia 20, às 9 horas, participar da audiência pública da Comissão de Segurança Pública e Combate ao Crime Organizado, com o tema Situação da Perícia Oficial no Brasil, no Plenário 6 do Anexo II.\r\n\tAtualmente, os peritos oficiais enfrentam 2 grandes problemas: a falta de regulamentação da carreira em nível federal e os baixos investimentos dos Governos Estaduais. Hoje, a discussão em torno da autonomia científica e funcional da perícia marca os principais debates. \r\n\tÉ sabido que a falta de autonomia desses profissionais tem causado sérios transtornos, e os casos de abusos de poder têm sido constantemente noticiados pela imprensa.\r\n\tA independência dos peritos já é realidade em 18 Estados do País, inclusive no meu, Mato Grosso. A importância desses profissionais é incontestável. S

In [ ]:
texto[0]

'Muito obrigado, Sr. Presidente. Somos coirmãos das cidades de Petrolina, Floresta, Afrânio e de todas as cidades vizinhas. Parabéns a V.Exa., que hoje está à frente da Presidência!\r\n\tSr. Presidente, Sras. e Srs. Deputados, venho a esta tribuna falar sobre a segurança pública do meu e seu, Presidente, Estado de Pernambuco.\r\n\tHá mais de 1 ano que o Governador Paulo Câmara assumiu o Estado de Pernambuco e deu continuidade ao programa de combate à violência Pacto Pela Vida, criado pelo saudoso Eduardo Campos, porém sem muito êxito, pois, somente no ano passado, a média foi de mais de 10 assassinatos por dia no Estado - na maioria deles, jovens foram as vítimas.\r\n\tA cidade de Afrânio, que está na divisa do Estado de Pernambuco com o Piauí e a Bahia, tem o mesmo número de policiais de 50 anos atrás, ou seja, apenas 3 policiais militares e policiais 2 civis, além de duas viaturas velhas, em condições precárias e sucateadas, apesar de atualmente ter população de 20 mil habitantes. Af

In [ ]:
type(texto)

pandas.core.series.Series

In [ ]:
entidades=[]
for idx,texto in enumerate(debate_dep['discurso_trat']):
    doc = nlp(texto)
    for entidade in doc.ents:
      entidades.append([idx,entidade.text,entidade.label_])


In [ ]:
dep_idx = []
for idx, id in enumerate(debate_dep['id_Deputado']):
  dep_idx.append([idx,id])

In [ ]:
df_dep_idx = pd.DataFrame(dep_idx, columns=["discurso","id_Deputado"])

In [ ]:
df_entidades = pd.DataFrame(entidades, columns=["discurso","entidade","rotulo"])

In [ ]:
id_dep.shape

(516, 6)

In [ ]:
id_dep = pd.merge(df_dep_idx[["id_Deputado",'discurso']], deputados[["id",'nome','siglaPartido','siglaUf']], left_on='id_Deputado', right_on='id', how="inner")

In [ ]:
import numpy as np
id_dep=id_dep.drop_duplicates()

In [ ]:
df_entidades.head(5)

,discurso,entidade,rotulo
0,0,Sr. Presidente,PER
1,0,Petrolina,LOC
2,0,Floresta,LOC
3,0,Afrânio,LOC
4,0,V.Exa,ORG


In [ ]:
ent_dep = pd.merge( df_entidades[["discurso",'entidade','rotulo']],id_dep[["discurso",'nome','siglaPartido', 'siglaUf']], left_on='discurso', right_on='discurso', how="inner")
ent_dep.shape

(25455, 6)

In [ ]:
ent_dep.head(5)

,discurso,entidade,rotulo,nome,siglaPartido,siglaUf
0,0,Sr. Presidente,PER,ADALBERTO CAVALCANTI,AVANTE,PE
1,0,Petrolina,LOC,ADALBERTO CAVALCANTI,AVANTE,PE
2,0,Floresta,LOC,ADALBERTO CAVALCANTI,AVANTE,PE
3,0,Afrânio,LOC,ADALBERTO CAVALCANTI,AVANTE,PE
4,0,V.Exa,ORG,ADALBERTO CAVALCANTI,AVANTE,PE


In [ ]:
# Lista de entidades a serem filtradas
entidades_a_filtrar = ["V.Exa","Sr. Presidente", "Srs", "Sras", "Tenho","Quero","Sr.","S.Exa","Deixo", "Fui", "acredito", "Agradeço"]
# Criando o filtro
filtro = ent_dep['entidade'].str.contains('|'.join(entidades_a_filtrar), na=False, case=False)
entidade_dep = ent_dep[~filtro]


In [ ]:
entidade_dep.shape

(21403, 6)

In [ ]:
entidade_dep['entidade'].value_counts()

País                                         1208
Brasil                                       1016
Estado                                        883
Deputados                                     632
Rio de Janeiro                                320
                                             ... 
Marcelo Belinati                                1
Ficam                                           1
Votei                                           1
Semana Santa                                    1
Secretaria de Segurança Pública do Estado       1
Name: entidade, Length: 2566, dtype: int64

In [ ]:
# Calcula a frequência de cada entidade
freq = entidade_dep['entidade'].value_counts()

# reita entidades que aparecem em menos de 10 discursos
entidades_frequentes = freq[freq >= 10].index

# retira entidades que aparecem em mais de 80% dos discursos
limite = 0.8 * len(datasetDiscursos)
entidades_raras = freq[freq <= limite].index

# Filtra o DataFrame usando os índices das entidades frequentes e raras
dataset_filtrado = entidade_dep[entidade_dep['entidade'].isin(entidades_frequentes)& entidade_dep['entidade'].isin(entidades_raras)]

In [ ]:
dataset_filtrado['entidade'].value_counts()

União                   107
Parlamento              106
Presidente              104
Capital                 102
Câmara dos Deputados     99
                       ... 
Ivo Cassol               10
Aripuanã                 10
Cultura - OEI            10
Guaíra                   10
PCdoB                    10
Name: entidade, Length: 336, dtype: int64

In [ ]:
dataset_filtrado.shape

(8354, 6)

In [ ]:
dataset_filtrado.head(5)

,discurso,entidade,rotulo,nome,siglaPartido,siglaUf
9,0,Presidente,MISC,ADALBERTO CAVALCANTI,AVANTE,PE
18,0,Piauí,LOC,ADALBERTO CAVALCANTI,AVANTE,PE
19,0,Bahia,LOC,ADALBERTO CAVALCANTI,AVANTE,PE
26,0,Governo do Estado,LOC,ADALBERTO CAVALCANTI,AVANTE,PE
28,0,Pernambuco,LOC,ADALBERTO CAVALCANTI,AVANTE,PE


In [ ]:
import matplotlib.pyplot as plt
# Contagem dos rótulos
rotulo_count = dataset_filtrado['rotulo'].value_counts()

# Cores diferentes para cada barra
cores = plt.cm.viridis(np.linspace(0, 1, len(rotulo_count)))

# Criar uma figura com tamanho personalizado
plt.figure(figsize=(12, 7))


# Gráfico de barras com cores diferentes
ax = rotulo_count.plot(kind='bar', color=cores)
plt.title('Contagem de Rótulos')
plt.xlabel('Rótulo')
plt.ylabel('Contagem')

# Adicionar os valores em cada barra
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.tight_layout()

# Salvando a figura em formato JPEG
plt.savefig("rotulos", format='jpeg', dpi=300)

plt.close()


In [ ]:
from wordcloud import WordCloud

# Contagem dos entidade
entidade_count = dataset_filtrado['entidade'].value_counts()

# Criando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate_from_frequencies(entidade_count)

# Mostrando a imagem
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Nuvem de Palavras - Rótulos')
plt.tight_layout()

# Salvando a figura em formato JPEG
plt.savefig("Nuvem_palavras", format='jpeg', dpi=300)

plt.close()


In [ ]:

# Filtrar o DataFrame com base no rótulo "Localidade"
df_localidade = dataset_filtrado[dataset_filtrado['rotulo'] == 'LOC']

# Contagem dos entidade
entidade_count = df_localidade['entidade'].value_counts()

# Criando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate_from_frequencies(entidade_count)

# Mostrando a imagem
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Nuvem de Palavras - Localidade')
plt.tight_layout()

# Salvando a figura em formato JPEG
plt.savefig("Nuvem_palavras_Loc", format='jpeg', dpi=300)

plt.close()

In [ ]:
# Filtrar o DataFrame com base no rótulo "Localidade"
df_pessoa = dataset_filtrado[dataset_filtrado['rotulo'] == 'PER']

# Contagem dos entidade
entidade_count = df_pessoa['entidade'].value_counts()

# Criando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate_from_frequencies(entidade_count)

# Mostrando a imagem
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Nuvem de Palavras - Pessoas')
plt.tight_layout()

# Salvando a figura em formato JPEG
plt.savefig("Nuvem_palavras_PER", format='jpeg', dpi=300)

plt.close()

In [ ]:
# Filtrar o DataFrame com base no rótulo "Localidade"
df_pessoa = dataset_filtrado[dataset_filtrado['rotulo'] == 'ORG']

# Contagem dos entidade
entidade_count = df_pessoa['entidade'].value_counts()

# Criando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate_from_frequencies(entidade_count)

# Mostrando a imagem
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Nuvem de Palavras - Oreganização')
plt.tight_layout()

# Salvando a figura em formato JPEG
plt.savefig("Nuvem_palavras_ORG", format='jpeg', dpi=300)

plt.close()


In [ ]:
# Contar a frequência de cada rótulo por partido
rotulos_por_partido = dataset_filtrado.groupby('siglaPartido')['rotulo'].value_counts()

# Calcular a porcentagem relativa
rotulos_percentagem = round(rotulos_por_partido.groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack().fillna(0),2)

# Organizar os resultados
rotulos_percentagem = rotulos_percentagem.sort_values(by='LOC', ascending=False)  # Aqui estou supondo que 'PER' é um dos rótulos. Se não for o caso, substitua por um dos seus rótulos.

# Exportar a tabela para um arquivo CSV

rotulos_percentagem.to_csv("rotulos_percentagem.csv")

rotulos_percentagem

<ipython-input-92-1adc1ab24729>:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rotulos_percentagem = round(rotulos_por_partido.groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack().fillna(0),2)


rotulo,LOC,MISC,ORG,PER
siglaPartido,,,,
PSD,90.00,4.29,2.14,3.57
PMN,84.62,7.69,0.00,7.69
PTdoB,72.73,0.00,9.09,18.18
PODE,71.30,12.17,7.83,8.70
PHS,70.93,8.14,12.79,8.14
PRB,68.48,10.85,12.40,8.27
REDE,66.67,16.67,16.67,0.00
PROS,65.53,6.60,14.67,13.20
PV,64.97,13.61,14.29,7.14


In [ ]:
# Calculando a porcentagem relativa
porcentagem_rotulo = round((contagem_rotulo / contagem_rotulo.sum()) * 100,2)
porcentagem_rotulo

LOC     66.28
PER     14.73
MISC     9.65
ORG      9.34
Name: rotulo, dtype: float64

In [ ]:
contagem_siglaPartido

PT               3950
PSDB             3214
PDT              2614
PMDB             1800
PSB              1389
PV               1132
MDB              1079
PSC               888
PPS               837
DEM               823
PRB               811
PP                748
PP**              737
PR                672
PROS              651
SDD               629
SOLIDARIEDADE     420
PCdoB             412
PTB               391
PSD               308
AVANTE            272
PSL               226
PHS               176
PODE              174
PMN               108
PSOL               99
PTdoB              56
REDE               18
Name: siglaPartido, dtype: int64

In [ ]:
contagem_siglaUf

BA    3525
PA    2884
SP    2268
RJ    2254
RS    2015
RO    1946
MG    1939
MT    1867
DF    1143
PR    1011
CE     686
MA     660
GO     626
PE     590
PB     415
ES     247
AC     217
AL     194
AM      43
PI      38
SC      37
TO      18
SE      11
Name: siglaUf, dtype: int64

In [ ]:
pd.crosstab(dataset_filtrado['entidade'], dataset_filtrado['rotulo'])

rotulo,LOC,MISC,ORG,PER
entidade,,,,
A Câmara,0,18,0,0
A Câmara dos Deputados,0,18,0,0
A Voz do Brasil,0,24,0,0
ALFREDO KAEFER,0,0,0,18
ALFREDO KAEFER - Pois,0,0,0,18
...,...,...,...,...
poliéster,0,27,0,0
§,0,0,37,0
Ásia,10,0,0,0


In [ ]:
# Contar a frequência de cada rótulo por partido
rotulos_por_partido = dataset_filtrado.groupby('siglaPartido')['rotulo'].value_counts()

# Calcular a porcentagem relativa
rotulos_percentagem = rotulos_por_partido.groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack().fillna(0)

# Organizar os resultados
rotulos_percentagem = rotulos_percentagem.sort_values(by='PER', ascending=False)  # Aqui estou supondo que 'PER' é um dos rótulos. Se não for o caso, substitua por um dos seus rótulos.
rotulos_percentagem

<ipython-input-48-1db8df1bd97c>:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rotulos_percentagem = rotulos_por_partido.groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack().fillna(0)


rotulo,LOC,MISC,ORG,PER
siglaPartido,,,,
PT,37.726218,13.503480,19.675174,29.095128
PTdoB,63.636364,0.000000,9.090909,27.272727
PSC,46.745562,13.609467,12.426036,27.218935
MDB,47.154472,15.447154,12.845528,24.552846
PP,53.670886,9.873418,12.151899,24.303797
PMDB,40.814196,20.668058,14.822547,23.695198
PSDB,50.134048,18.163539,8.914209,22.788204
SOLIDARIEDADE,50.666667,14.666667,12.000000,22.666667
DEM,52.882206,14.285714,12.781955,20.050125


In [ ]:
filtro = entidade_dep['entidade'].str.contains("Sr. Presidente", na=False, case=False)
df_entidades = df_entidades[~filtro]

In [ ]:
# Calcula a frequência de cada entidade
freq = entidade_dep['entidade'].value_counts().head(25)
freq
# Filtra entidades que aparecem em menos de 10 discursos
#entidades_frequentes = freq[freq >= 10].index

# Filtra entidades que aparecem em mais de 50% dos discursos
#limite = 0.8 * len(datasetDiscursos)
#entidades_raras = freq[freq <= limite].index

# Filtra o DataFrame usando os índices das entidades frequentes e raras
#dataset_filtrado = entidade_dep[entidade_dep['entidade'].isin(entidades_frequentes) & entidade_dep['entidade'].isin(entidades_raras)]

País                  1970
Brasil                1550
Estado                1357
Sr. Presidente        1249
Deputados              969
Srs                    958
Sras                   846
Rio de Janeiro         488
Estados                433
Governo                408
Governo Federal        386
Congresso Nacional     367
Casa                   364
Deputado               255
Municípios             242
Polícia Federal        221
Amazônia               218
Tenho                  200
Parlamentares          191
Rondônia               182
Polícia                178
Pará                   176
União                  170
Ministério Público     167
Presidente             167
Name: entidade, dtype: int64

In [ ]:
nlp = spacy.load("pt_core_news_sm")
# raw_text="O SR. ABILIO BRUNINI (PL - MT. Para encaminhar. Sem revisão do orador.) - Sr. Presidente, não sei se todos estão entendendo, mas, no dia de hoje, o Governo Federal está cancelando investimentos de 1 milhão e 750 mil do Programa de Proteção do Direito à Vida. Cadê o pessoal que defende a vida, quando 1 milhão e 750 mil estão sendo cancelados da proteção do direito à vida? Outra questão: 8 milhões estão sendo cancelados do Ministério Público. É o enfraquecimento do Ministério Público! Oito milhões estão sendo retirados do Ministério Público! Mais uma: 3,5 milhões de reais estão sendo cancelados da Controladoria-Geral da União. Isso significa o enfraquecimento da Controladoria-Geral da União &#x97; 3,5 milhões estão sendo cancelados! Vejam a área da cultura! Pessoal da cultura, esquerdistas: 6,7 milhões de reais estão sendo cancelados da promoção e do fomento à cultura brasileira nacional. Repito: 6,7 milhões de reais estão sendo cancelados. Instituto Chico Mendes: 2,9 milhões de reais estão sendo cancelados do Instituto Chico Mendes! Agricultura familiar: 6,5 milhões de reais estão sendo cancelados da agricultura familiar! Mais: 25 milhões de reais estão sendo cancelados dos programas de combate à fome! Em relação à assistência social, 7,6 milhões de reais estão sendo cancelados! Fortalecimento da família: 3,6 milhões de reais estão sendo cancelados! Cadê o amor? Cadê o pessoal da cultura? Cadê o PSOL? Cadê o PT? Cadê os puxa-sacos do Lula? Vocês não reclamam?! São 25 milhões de reais do combate à fome que estão sendo cancelados! Cadê o pessoal do Nordeste? O Governo Federal está cancelando 25 milhões de reais do programa de combate à fome. Cadê o pessoal do Nordeste que não reclama, mesmo com a miséria que há por lá? São 6,5 milhões de reais que estão sendo cancelados da agricultura familiar! Cadê o MST? Cadê o pessoal do MST? Estão cancelando 6,5 milhões de reais da agricultura familiar! Estão cancelando 2,9 milhões de reais das organizações sociais! Cadê o pessoal que defende as ONGs? Estão sendo cancelados 2,9 milhões da assistência social e das organizações sociais! Viva a hipocrisia! Eu nem estou falando da Universidade Federal de Santa Maria, que está perdendo 4 milhões. Nem estou falando da Universidade Federal do Rio de Janeiro, que está perdendo 21 milhões. Nem estou falando da Universidade Federal do Rio Grande do Norte, que está perdendo 8 milhões. Não estou nem falando da Universidade Federal de Roraima, que está perdendo 2,3 milhões; do Centro Federal de Educação Tecnológica de Minas Gerais, que está perdendo 989 mil; da Universidade Federal de Alfenas, que está perdendo 530 mil; da Universidade Federal de São Paulo, que está perdendo 952 mil; da Fundação da Universidade Federal de Pampa, que está perdendo 92 mil; da Fundação Universidade do Rio de Janeiro, que está perdendo quase 400 mil; da Fundação Universidade da Amazônia, que está perdendo 490 mil! Cadê o pessoal da educação? Viva a hipocrisia! Eu vim para cobrar. Meu nome é Abilio!"
raw_text=texto
doc = nlp(raw_text)

print([d for d in doc])

# tokenizing doc object
[token.text for token in doc]

[O, SR, ., HUMBERTO, SOUTO, (, PPS-MG, ., Pronuncia, o, seguinte, discurso, ., ), -, Sr., Presidente, ,, Sras, ., e, Srs, ., Deputados, ,, o, jornal, Estado, de, Minas, ,, de, Belo, Horizonte, ,, enriqueceu, a, comunicação, brasileira, ao, reforçar, o, seu, quadro, de, cronistas, com, o, sociólogo, e, cientista, político, Marcos, Coimbra, ,, respeitado, analista, de, pesquisas, ,, no, Brasil, e, até, no, exterior, ., 
, Os, comentários, do, ilustre, diretor, do, instituto, Vox, Populi, nas, páginas, do, Estado, de, Minas, certamente, vão, contribuir, para, o, melhor, entendimento, de, fatos, e, entrelinhas, de, pesquisas, que, apontam, tendências, ,, caminhos, e, conceitos, nacionais, ., 
, Portanto, ,, o, Estado, de, Minas, e, o, cientista, político, Marcos, Coimbra, presentearão, os, mineiros, ,, brasileiros, e, cidadãos, do, mundo, inteiro, com, informações, antes, restritas, a, poucos, ., Por, isso, ,, gostaria, que, a, Mesa, Diretora, da, Câmara, dos, Deputados, manifestasse, a, a

['O',
 'SR',
 '.',
 'HUMBERTO',
 'SOUTO',
 '(',
 'PPS-MG',
 '.',
 'Pronuncia',
 'o',
 'seguinte',
 'discurso',
 '.',
 ')',
 '-',
 'Sr.',
 'Presidente',
 ',',
 'Sras',
 '.',
 'e',
 'Srs',
 '.',
 'Deputados',
 ',',
 'o',
 'jornal',
 'Estado',
 'de',
 'Minas',
 ',',
 'de',
 'Belo',
 'Horizonte',
 ',',
 'enriqueceu',
 'a',
 'comunicação',
 'brasileira',
 'ao',
 'reforçar',
 'o',
 'seu',
 'quadro',
 'de',
 'cronistas',
 'com',
 'o',
 'sociólogo',
 'e',
 'cientista',
 'político',
 'Marcos',
 'Coimbra',
 ',',
 'respeitado',
 'analista',
 'de',
 'pesquisas',
 ',',
 'no',
 'Brasil',
 'e',
 'até',
 'no',
 'exterior',
 '.',
 '\r\n',
 'Os',
 'comentários',
 'do',
 'ilustre',
 'diretor',
 'do',
 'instituto',
 'Vox',
 'Populi',
 'nas',
 'páginas',
 'do',
 'Estado',
 'de',
 'Minas',
 'certamente',
 'vão',
 'contribuir',
 'para',
 'o',
 'melhor',
 'entendimento',
 'de',
 'fatos',
 'e',
 'entrelinhas',
 'de',
 'pesquisas',
 'que',
 'apontam',
 'tendências',
 ',',
 'caminhos',
 'e',
 'conceitos',
 'naci

In [ ]:
# attribute, example: text GOING, lemma: numbers, lemma: GO
for token in doc:
  print(token.text, token.lemma, token.lemma_)

O 1489474827855109852 o
SR 46538067201949991 SR
. 12646065887601541794 .
HUMBERTO 4860439877451293186 HUMBERTO
SOUTO 18189630044975325938 SOUTO
( 12638816674900267446 (
PPS-MG 11384412777850996552 PPS-MG
. 12646065887601541794 .
Pronuncia 15151082246175895404 Pronuncia
o 1489474827855109852 o
seguinte 11206845557231892115 seguinte
discurso 11068264254112188247 discurso
. 12646065887601541794 .
) 3842344029291005339 )
- 9153284864653046197 -
Sr. 13821695289612080669 sr.
Presidente 975269286897690711 presidente
, 2593208677638477497 ,
Sras 7416448108489228436 Sras
. 12646065887601541794 .
e 1720370409040345145 e
Srs 13652984843079274771 Srs
. 12646065887601541794 .
Deputados 6358773664443206815 deputado
, 2593208677638477497 ,
o 1489474827855109852 o
jornal 17774790523688002913 jornal
Estado 16787858920008161436 estado
de 11144093025662894627 de
Minas 3344967053525537263 Minas
, 2593208677638477497 ,
de 11144093025662894627 de
Belo 8756349092843369845 Belo
Horizonte 10404466369148261211 

In [ ]:
# viewing sentece's structure
from spacy import displacy
from IPython.display import SVG, display
def showSVG(s):
  display(SVG(s))

graph01 = displacy.render(doc)
showSVG(graph01)

In [ ]:
# IS ALPHA, IS PUNCT, IS LOWER.. etc
for token in doc:
  print('TXT:',token.text, ' -- é caracter?:',token.is_alpha,
        'é pontuacao?:', token.is_punct, 'está em caixa baixa?:', token.is_lower)

TXT: O  -- é caracter?: True é pontuacao?: False está em caixa baixa?: False
TXT: SR  -- é caracter?: True é pontuacao?: False está em caixa baixa?: False
TXT: .  -- é caracter?: False é pontuacao?: True está em caixa baixa?: False
TXT: HUMBERTO  -- é caracter?: True é pontuacao?: False está em caixa baixa?: False
TXT: SOUTO  -- é caracter?: True é pontuacao?: False está em caixa baixa?: False
TXT: (  -- é caracter?: False é pontuacao?: True está em caixa baixa?: False
TXT: PPS-MG  -- é caracter?: False é pontuacao?: False está em caixa baixa?: False
TXT: .  -- é caracter?: False é pontuacao?: True está em caixa baixa?: False
TXT: Pronuncia  -- é caracter?: True é pontuacao?: False está em caixa baixa?: False
TXT: o  -- é caracter?: True é pontuacao?: False está em caixa baixa?: True
TXT: seguinte  -- é caracter?: True é pontuacao?: False está em caixa baixa?: True
TXT: discurso  -- é caracter?: True é pontuacao?: False está em caixa baixa?: True
TXT: .  -- é caracter?: False é pontuac

In [ ]:
# structure POS (Part-of-Speech)
[(token.text, token.pos, token.pos_) for token in doc]

[('O', 90, 'DET'),
 ('SR', 96, 'PROPN'),
 ('.', 97, 'PUNCT'),
 ('HUMBERTO', 96, 'PROPN'),
 ('SOUTO', 96, 'PROPN'),
 ('(', 97, 'PUNCT'),
 ('PPS-MG', 96, 'PROPN'),
 ('.', 97, 'PUNCT'),
 ('Pronuncia', 96, 'PROPN'),
 ('o', 90, 'DET'),
 ('seguinte', 84, 'ADJ'),
 ('discurso', 92, 'NOUN'),
 ('.', 97, 'PUNCT'),
 (')', 97, 'PUNCT'),
 ('-', 97, 'PUNCT'),
 ('Sr.', 85, 'ADP'),
 ('Presidente', 92, 'NOUN'),
 (',', 97, 'PUNCT'),
 ('Sras', 96, 'PROPN'),
 ('.', 97, 'PUNCT'),
 ('e', 89, 'CCONJ'),
 ('Srs', 96, 'PROPN'),
 ('.', 97, 'PUNCT'),
 ('Deputados', 96, 'PROPN'),
 (',', 97, 'PUNCT'),
 ('o', 90, 'DET'),
 ('jornal', 92, 'NOUN'),
 ('Estado', 100, 'VERB'),
 ('de', 85, 'ADP'),
 ('Minas', 96, 'PROPN'),
 (',', 97, 'PUNCT'),
 ('de', 85, 'ADP'),
 ('Belo', 96, 'PROPN'),
 ('Horizonte', 96, 'PROPN'),
 (',', 97, 'PUNCT'),
 ('enriqueceu', 100, 'VERB'),
 ('a', 90, 'DET'),
 ('comunicação', 92, 'NOUN'),
 ('brasileira', 84, 'ADJ'),
 ('ao', 98, 'SCONJ'),
 ('reforçar', 100, 'VERB'),
 ('o', 90, 'DET'),
 ('seu', 90, 'DE

In [ ]:
# return tokens that pos_ is VERB
[(token.text, token.lemma_, token.pos_) for token in doc]# if token.pos_ == 'VERB']

[('O', 'o', 'DET'),
 ('SR', 'SR', 'PROPN'),
 ('.', '.', 'PUNCT'),
 ('HUMBERTO', 'HUMBERTO', 'PROPN'),
 ('SOUTO', 'SOUTO', 'PROPN'),
 ('(', '(', 'PUNCT'),
 ('PPS-MG', 'PPS-MG', 'PROPN'),
 ('.', '.', 'PUNCT'),
 ('Pronuncia', 'Pronuncia', 'PROPN'),
 ('o', 'o', 'DET'),
 ('seguinte', 'seguinte', 'ADJ'),
 ('discurso', 'discurso', 'NOUN'),
 ('.', '.', 'PUNCT'),
 (')', ')', 'PUNCT'),
 ('-', '-', 'PUNCT'),
 ('Sr.', 'sr.', 'ADP'),
 ('Presidente', 'presidente', 'NOUN'),
 (',', ',', 'PUNCT'),
 ('Sras', 'Sras', 'PROPN'),
 ('.', '.', 'PUNCT'),
 ('e', 'e', 'CCONJ'),
 ('Srs', 'Srs', 'PROPN'),
 ('.', '.', 'PUNCT'),
 ('Deputados', 'deputado', 'PROPN'),
 (',', ',', 'PUNCT'),
 ('o', 'o', 'DET'),
 ('jornal', 'jornal', 'NOUN'),
 ('Estado', 'estado', 'VERB'),
 ('de', 'de', 'ADP'),
 ('Minas', 'Minas', 'PROPN'),
 (',', ',', 'PUNCT'),
 ('de', 'de', 'ADP'),
 ('Belo', 'Belo', 'PROPN'),
 ('Horizonte', 'Horizonte', 'PROPN'),
 (',', ',', 'PUNCT'),
 ('enriqueceu', 'enriquecer', 'VERB'),
 ('a', 'o', 'DET'),
 ('comunic

In [ ]:
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

SR 2 4 MISC
SOUTO 15 20 MISC
PPS-MG 22 28 ORG
Sr. Presidente 64 78 PER
Sras 80 84 PER
Srs 88 91 LOC
Deputados 93 102 PER
Estado de Minas 113 128 LOC
Belo Horizonte 133 147 LOC
Marcos Coimbra 260 274 PER
Brasil 313 319 LOC
instituto Vox Populi 377 397 ORG
Estado de Minas 413 428 LOC
Estado de Minas 587 602 LOC
Marcos Coimbra 626 640 PER
Mesa Diretora da Câmara dos Deputados 774 811 LOC
Pesquisas 941 950 LOC
Marcos Coimbra 973 987 PER
Pesquisa Datafolha 1033 1051 PER
instituto Vox Populi 1190 1210 ORG
Governo 1225 1232 LOC
Luiz Inácio Lula da  Silva 1247 1273 PER
País 1291 1295 LOC
Alerta Marcos Coimbra 1371 1392 PER
João Hélio 1631 1641 PER
Rio de Janeiro 1646 1660 LOC
Revela 1663 1669 LOC
Datafolha 1718 1727 LOC
Vox Populi 1784 1794 ORG
Governo Lula 1889 1901 ORG
Capitais 2000 2008 LOC
Marcos Coimbra 2348 2362 PER
Assaltos 2455 2463 PER
Montes Claros 2609 2622 LOC
Minas 2636 2641 LOC
Montes Claros 2732 2745 LOC
Polícias Militar 2750 2766 LOC
Civil 2769 2774 LOC
Montes Claros 3174 3187 

In [ ]:
from collections import Counter

In [ ]:
import pandas as pd
pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400

In [ ]:
people = []

for named_entity in doc.ents:
    # if named_entity.label_ == "PER":
    people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
df

# for document in chunked_documents:
#     for named_entity in document.ents:
#         if named_entity.label_ == "PER":
#             people.append(named_entity.text)

# people_tally = Counter(people)

# df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
# df

,ent.label_,count
0,Marcos Coimbra,5
1,Governo Federal,4
2,Sr. Presidente,3
3,Estado de Minas,3
4,Governo,3
5,Montes Claros,3
6,Minas,3
7,Sras,2
8,Srs,2
9,Deputados,2
